In [4]:
import sqlite3
import csv
import pandas as pd
from datetime import datetime

In [5]:
def read_sales_csv(filename):
    sales = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            date = datetime.strptime(row['date'], '%m/%d/%y')
            sales.append({'date': date,
                     'days_ago': (datetime.now() - date).days,
                     'account_number': row['account_number'],
                     'price': int(row['price'])})
    return sales

conn = sqlite3.connect('file:NW_Central_OKC_w_bldg_details.db?mode=ro', uri=True)
c = conn.cursor()

sales = read_sales_csv('sales_list_2019.csv')
#sales = sales + read_sales_csv('sales_list_2018.csv')
#sales = sales + read_sales_csv('sales_list_2017.csv')

Now let's load the data into DataFrames. We have two tables we're using, one for properties/parcels and one for buildings. So what we'll do is pull out some data about the buildings, sum it up (as some properties have more than 1 building), and add it to the property DF.

In [27]:
combined_df = pd.read_sql_query("SELECT * FROM realproperty INNER JOIN buildings ON realproperty.id = buildings.local_property_id;", conn)
rp_df = pd.read_sql_query("SELECT * FROM realproperty WHERE property_type = 'Residential'", conn)
rp_df = rp_df.set_index('account_number')

# Get data about the buildings for each parcel. We'll sum up the data for now
# (in cases where there's >1 building on the property)
rp_df['sqft_sum'] = combined_df.groupby(['account_number'])['sq_ft'].sum()
rp_df['bed_sum'] = combined_df.replace(-1,0).groupby(['account_number'])['bedrooms'].sum()
rp_df['bath_sum'] = combined_df.replace(-1,0).groupby(['account_number'])['full_bathrooms'].sum() +\
    (combined_df.groupby(['account_number'])['three_quarters_bathrooms'].sum() * 0.75) +\
    (combined_df.groupby(['account_number'])['half_bathrooms'].sum() * 0.5)

display(rp_df)

,id,propertyid,property_type,location,building_name_occupant,city,owner_name_1,owner_name_2,billing_address_1,billing_address_2,...,lot_depth,land_value,quarter_section_description,subdivision,block,lot,legal_description,sqft_sum,bed_sum,bath_sum
account_number,,,,,,,,,,,,,,,,,,,,,
R049810025,1,130116,Residential,400 NW 43RD ST,,OKLAHOMA CITY,YOUNGS STEVEN ROGERS,TURLEY ALLISON,400 NW 43RD ST,,...,None,52721,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,001,EDGEMERE TERRACE 001 001,1824.0,3.0,2.00
R049810050,3,130117,Residential,408 NW 43RD ST,,OKLAHOMA CITY,JUNGMAN TIMOTHY WILLIAM & JULIE LYN,,408 NW 43RD ST,,...,None,45900,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,002,EDGEMERE TERRACE 001 002,2677.0,3.0,2.50
R049480080,6,130504,Residential,1009 NW 42ND ST,,OKLAHOMA CITY,SPEARS KENNETH L & REGINA,,210 NW 10TH ST,,...,None,71460,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,004,CROWN HEIGHTS ADD 001 004,1961.0,2.0,2.00
R049640005,7,131004,Residential,5021 N WALKER AVE,,OKLAHOMA CITY,CRISP STEVE & ANGIE,,5021 N WALKER,,...,None,48230,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 1 & 2,1352.0,3.0,1.50
R049640010,8,131005,Residential,504 NW 50TH ST,,OKLAHOMA CITY,DICKSON JERRY ROBERT,,504 NW 50TH ST,,...,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 3 & 4,1386.0,3.0,1.50
R049640015,9,131006,Residential,508 NW 50TH ST,,OKLAHOMA CITY,GARRETT & COMPANY LLC,,9701 BROADWAY EXT,,...,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 5 & 6,1092.0,2.0,1.00
R049810100,10,130119,Residential,418 NW 43RD ST,,OKLAHOMA CITY,BRADEN THOMAS GARY & BEVERLY JOAN,REVOCABLE LIVING TRUST,418 NW 43RD ST,,...,None,45900,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,004,EDGEMERE TERRACE 001 004,2047.0,2.0,2.00
R049640020,12,131007,Residential,512 NW 50TH ST,,OKLAHOMA CITY,HALPIN JEFFREY J,,512 NW 50TH ST,,...,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 7 & 8,1038.0,2.0,1.00
R049480100,13,130505,Residential,1005 NW 42ND ST,,OKLAHOMA CITY,DK INC,,C/O KENNETH L SPEARS,PO BOX 687,...,None,71460,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,005,CROWN HEIGHTS ADD 001 005,1660.0,2.0,2.00


In [28]:
# This filters for subdivision based on number of properties in that sub. Let's change it
# later to filter for number of transactions.
sublist = (rp_df[['subdivision']]
            .assign(count = 0)
            .groupby("subdivision")
            .count())
sublist = sublist.where(sublist['count'] >= 100).dropna()
sublist_names = sublist.index.values
display(sublist_names)

rp_df['subdivision'] = rp_df.apply(lambda x: 'NOT LISTED' if x['subdivision'] not in sublist_names else x['subdivision'], axis=1)

array(['AURORA ADDITION', 'BAUMANS ADDITION', 'BELL VERN ADDITION',
       'CARLE & COLCORD ADDITION', 'CASHION PLACE ADDITION',
       'CASHION PLACE AMD', 'CENTRAL PARK ADDITION',
       'CLASSENS CREAM RIDGE', 'COLLEGE ADDITION', 'CRESTWOOD ADDITION',
       'CROWN HEIGHTS ADD', 'DARRALLS SECOND ADD', 'DENNISTON PARK ADD',
       'DOUGLAS PLACE ADD', 'EDGEMERE PARK ADD', 'EDGEMERE TERRACE',
       'EPWORTH VIEW ADD', 'GARDEN ADDITION', 'GATEWOOD ADDITION',
       'GRAND VIEW ADDITION', 'GUERNSEY PARK PLACE',
       'HEMINGWAY CONDOMINIUMS', 'INGLESIDE TO OKLA CITY',
       'JEFFERSON PARK ADD', 'LAS VEGAS ADDITION',
       'LINWOOD PLACE AMENDED', 'LINWOOD PLACE SECOND',
       'LYONS WILL ROGERS PK', 'MAYFAIR HEIGHTS BLKS 1 THRU 29',
       'MILAM PLACE ADDITION', 'MILITARY PARK ADD', 'MILLERS BOULEVARD',
       'MORRIS WILL ROGER PARK', 'NEAS ADDITION', 'PACKINGHOUSE PARK',
       'PARKER & COLCORD ADD', 'PUTNAM HEIGHTS 2ND', 'PUTNAM HEIGHTS ADD',
       'RAVENSWOOD ADDITION', 'RI

In [29]:
sales_df = pd.DataFrame(sales)
display(sales_df)

,account_number,date,days_ago,price
0,R188308300,2019-10-10,4,216000
1,R212841600,2019-10-10,4,227000
2,R109711175,2019-10-09,5,75000
3,R109731255,2019-10-09,5,120000
4,R151661260,2019-10-09,5,49000
5,R056651065,2019-10-09,5,65000
6,R122251185,2019-10-09,5,86000
7,R130762112,2019-10-08,6,40500
8,R203061190,2019-10-08,6,825000
9,R034354750,2019-10-08,6,140000


In [30]:
merged_df = pd.merge(rp_df, sales_df, on='account_number', how='inner')
display(merged_df)

,account_number,id,propertyid,property_type,location,building_name_occupant,city,owner_name_1,owner_name_2,billing_address_1,...,subdivision,block,lot,legal_description,sqft_sum,bed_sum,bath_sum,date,days_ago,price
0,R049640005,7,131004,Residential,5021 N WALKER AVE,,OKLAHOMA CITY,CRISP STEVE & ANGIE,,5021 N WALKER,...,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 1 & 2,1352.0,3.0,1.50,2019-07-02,104,108000
1,R049640005,7,131004,Residential,5021 N WALKER AVE,,OKLAHOMA CITY,CRISP STEVE & ANGIE,,5021 N WALKER,...,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 1 & 2,1352.0,3.0,1.50,2019-05-10,157,108000
2,R049810175,27,130122,Residential,436 NW 43RD ST,,OKLAHOMA CITY,BELFLOWER ADAM P & ALEXANDRA,,436 NW 43RD ST,...,EDGEMERE TERRACE,001,007,EDGEMERE TERRACE 001 007,2018.0,3.0,2.00,2019-04-05,192,305000
3,R049640070,50,131015,Residential,5000 N LEE AVE,,OKLAHOMA CITY,KUMBALEK MICHAEL MAX,,5000 N LEE AVE,...,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 23 & 24,1429.0,2.0,1.75,2019-05-15,152,157000
4,R049810375,66,130130,Residential,427 NW 43RD ST,,OKLAHOMA CITY,STALL JOSPEH E,LAWMASTER JENNIFER L,427 NW 43RD ST,...,EDGEMERE TERRACE,002,005,EDGEMERE TERRACE 002 005,2155.0,3.0,2.00,2019-09-06,38,335000
5,R049810425,76,130132,Residential,415 NW 43RD ST,,OKLAHOMA CITY,SCHEER BRADY D,SCHEER MACKENZIE K,415 NW 43RD ST,...,EDGEMERE TERRACE,002,007,EDGEMERE TERRACE 002 007,2097.0,3.0,2.50,2019-06-05,131,343500
6,R049810425,76,130132,Residential,415 NW 43RD ST,,OKLAHOMA CITY,SCHEER BRADY D,SCHEER MACKENZIE K,415 NW 43RD ST,...,EDGEMERE TERRACE,002,007,EDGEMERE TERRACE 002 007,2097.0,3.0,2.50,2019-05-06,161,343500
7,R049820200,94,130383,Residential,305 NW 42ND ST,,OKLAHOMA CITY,SCAPERLANDA MICHAEL A & MARIA RUIZ,,3816 WAVERLY CT,...,NOT LISTED,001,002,EDGEMERE HEIGHTS ADD 001 002,2541.0,4.0,2.00,2019-05-29,138,415000
8,R049810550,101,130137,Residential,412 NW 44TH ST,,OKLAHOMA CITY,PENNINGTON PARKER B,,412 NW 44TH ST,...,EDGEMERE TERRACE,002,012,EDGEMERE TERRACE 002 012,1416.0,3.0,1.75,2019-03-14,214,182500
9,R049480480,111,130524,Residential,804 NW 42ND ST,,OKLAHOMA CITY,DOSSETT SEAN,DOSSETT ALYSSA,804 NW 42ND ST,...,CROWN HEIGHTS ADD,003,002,CROWN HEIGHTS ADD 003 002,2316.0,4.0,1.75,2019-06-13,123,555000


In [44]:
new_df = merged_df[['land_size', 'land_value', 'subdivision', 'sqft_sum', 'bed_sum', 'bath_sum', 'price', 'days_ago']]

display(new_df)

,land_size,land_value,subdivision,sqft_sum,bed_sum,bath_sum,price,days_ago
0,9147.6,48230,DOUGLAS PLACE ADD,1352.0,3.0,1.50,108000,104
1,9147.6,48230,DOUGLAS PLACE ADD,1352.0,3.0,1.50,108000,157
2,9147.6,45900,EDGEMERE TERRACE,2018.0,3.0,2.00,305000,192
3,6969.6,37100,DOUGLAS PLACE ADD,1429.0,2.0,1.75,157000,152
4,8276.4,42840,EDGEMERE TERRACE,2155.0,3.0,2.00,335000,38
5,8276.4,42840,EDGEMERE TERRACE,2097.0,3.0,2.50,343500,131
6,8276.4,42840,EDGEMERE TERRACE,2097.0,3.0,2.50,343500,161
7,17859.6,85603,NOT LISTED,2541.0,4.0,2.00,415000,138
8,7405.2,36720,EDGEMERE TERRACE,1416.0,3.0,1.75,182500,214
9,8276.4,80559,CROWN HEIGHTS ADD,2316.0,4.0,1.75,555000,123


In [45]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [46]:
le = preprocessing.LabelEncoder()
new_df['sub_encoded'] = le.fit_transform(new_df['subdivision'])
new_df = new_df.drop(columns=['subdivision']).dropna()
display(new_df)

X = new_df[['land_size', 'land_value', 'sub_encoded', 'sqft_sum', 'bed_sum', 'bath_sum', 'days_ago']]
y = new_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

/Users/scoot/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,land_size,land_value,sqft_sum,bed_sum,bath_sum,price,days_ago,sub_encoded
0,9147.6,48230,1352.0,3.0,1.50,108000,104,13
1,9147.6,48230,1352.0,3.0,1.50,108000,157,13
2,9147.6,45900,2018.0,3.0,2.00,305000,192,15
3,6969.6,37100,1429.0,2.0,1.75,157000,152,13
4,8276.4,42840,2155.0,3.0,2.00,335000,38,15
5,8276.4,42840,2097.0,3.0,2.50,343500,131,15
6,8276.4,42840,2097.0,3.0,2.50,343500,161,15
7,17859.6,85603,2541.0,4.0,2.00,415000,138,34
8,7405.2,36720,1416.0,3.0,1.75,182500,214,15
9,8276.4,80559,2316.0,4.0,1.75,555000,123,10


In [51]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)

In [52]:
knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [53]:
knn.score(X_test, y_test)

0.4712116266139997

In [71]:
test = knn.predict([[10000, 80000, 50, 3000, 5, 3, 10]])

In [72]:
test[0]

302800.0

# Linear Regression

Now let's try this with linear regression rather than KNN.

In [58]:
from sklearn.linear_model import LinearRegression
print(X_train)

linreg = LinearRegression().fit(X_train, y_train)

      land_size  land_value  sub_encoded  sqft_sum  bed_sum  bath_sum  \
851      7405.2       25050           34    1673.0      3.0      1.75   
848      7405.2       16575           34    1085.0      2.0      1.00   
97       6969.6       37100           13    1223.0      3.0      1.00   
328      6534.0       14973           40    1616.0      3.0      1.50   
940      6969.6       26040            9    1777.0      4.0      2.00   
1077    11325.6      131662           57    4758.0      5.0      3.50   
196      7840.8       23868           56    1445.0      3.0      1.75   
342      6098.4       10561           27     768.0      2.0      1.00   
183      8276.4       25260           56    1580.0      3.0      1.75   
1200     6969.6        1680           36    1315.0      3.0      2.00   
416      6969.6       14835           39    1080.0      2.0      2.00   
458      6969.6       18060           22    1356.0      1.0      1.00   
787      7405.2       22650            0    1399.0 

In [68]:
linreg.predict([[10000,80000,1,5000,4,3,10]])

array([622043.29213313])

In [69]:
linreg.score(X_test, y_test)

0.6504435662531397

# One Hot Encoding

Let's change the subdivision feature to one hot encoding

In [73]:
new_df = (merged_df[['land_size', 'land_value', 'subdivision', 'sqft_sum', 'bed_sum', 'bath_sum', 'days_ago', 'price']][merged_df.price != 0]
          .dropna()
          .reset_index()
          .drop(columns=['index']))
#new_df = new_df[new_df.price != 0]

ohenc = preprocessing.OneHotEncoder()
ohenc.fit([[x] for x in new_df['subdivision']])
ohenc.categories_
encoded = ohenc.transform([[x] for x in new_df['subdivision']]).toarray()
encoded_df = pd.DataFrame(encoded, columns=["Subdivision_"+str(i) for i in range(encoded.shape[1])])
#display(encoded_df)
#new_df = pd.get_dummies(new_df, prefix=['subdivision'])

new_df = new_df.drop(columns=['subdivision'])
new_df = pd.concat([new_df, encoded_df], axis=1)
display(new_df)

,land_size,land_value,sqft_sum,bed_sum,bath_sum,days_ago,price,Subdivision_0,Subdivision_1,Subdivision_2,...,Subdivision_49,Subdivision_50,Subdivision_51,Subdivision_52,Subdivision_53,Subdivision_54,Subdivision_55,Subdivision_56,Subdivision_57,Subdivision_58
0,9147.6,48230,1352.0,3.0,1.50,104,108000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9147.6,48230,1352.0,3.0,1.50,157,108000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9147.6,45900,2018.0,3.0,2.00,192,305000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6969.6,37100,1429.0,2.0,1.75,152,157000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8276.4,42840,2155.0,3.0,2.00,38,335000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,8276.4,42840,2097.0,3.0,2.50,131,343500,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,8276.4,42840,2097.0,3.0,2.50,161,343500,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,17859.6,85603,2541.0,4.0,2.00,138,415000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,7405.2,36720,1416.0,3.0,1.75,214,182500,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,8276.4,80559,2316.0,4.0,1.75,123,555000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
X = new_df.drop('price',axis=1)
y = new_df['price']
#display(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
linreg = LinearRegression().fit(X_train, y_train)
#display(X_test)
linreg.score(X_test, y_test)

0.6640636269832267

In [76]:
import numpy as np
ch_oh = ohenc.transform([["EDGEMERE TERRACE"]]).toarray()
ch_args = [10000, 40000, 3000, 4, 3, 10] + ch_oh.tolist()[0]
ch_args = [6400, 34790, 1500, 4, 3, 10] + ch_oh.tolist()[0]
#ch_args = ch_oh.tolist()
linreg.predict([ch_args])

array([216952.50628707])